## SF-crime Kaggle competition

In [1]:
# Let's start by some standard code
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from IPython.display import display

from sklearn import metrics
from graphviz import *

In [3]:
PATH = "data/sf-crime/"

In [4]:
!ls {PATH}

result_v1.csv        result_version1.csv  sampleSubmission.csv
result_v2.csv        result_version2.csv  test.csv
result_v3.csv        result_version3.csv  train.csv
result_v4.csv        result_version4.csv
result_v5.csv        result_version5.csv


### 1. Let's load and explore the data

In [15]:
train=pd.read_csv(f'{PATH}train.csv',parse_dates=['Dates'])

In [16]:
train.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')

In [17]:
display(train.head())

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [18]:
display(train.isnull().sum().sort_index()/len(train))

Address       0.0
Category      0.0
Dates         0.0
DayOfWeek     0.0
Descript      0.0
PdDistrict    0.0
Resolution    0.0
X             0.0
Y             0.0
dtype: float64

### What we see in the data
1. One date field. So the date formatting and feature extraction is required.
2. A few categorical variables. We will have to convert these into numbers.
3. No missing values.

### Let's explore the response variable (y)

In [19]:
train['Category'].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT', 'VANDALISM', 'NON-CRIMINAL',
       'ROBBERY', 'ASSAULT', 'WEAPON LAWS', 'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY', 'SECONDARY CODES', 'TRESPASS',
       'MISSING PERSON', 'FRAUD', 'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT', 'ARSON', 'FAMILY OFFENSES',
       'LIQUOR LAWS', 'BRIBERY', 'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS', 'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

In [20]:
train.groupby('Category').count().sort_values('Dates')

,Dates,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
Category,,,,,,,,
TREA,6,6,6,6,6,6,6,6
PORNOGRAPHY/OBSCENE MAT,22,22,22,22,22,22,22,22
GAMBLING,146,146,146,146,146,146,146,146
SEX OFFENSES NON FORCIBLE,148,148,148,148,148,148,148,148
EXTORTION,256,256,256,256,256,256,256,256
BRIBERY,289,289,289,289,289,289,289,289
BAD CHECKS,406,406,406,406,406,406,406,406
FAMILY OFFENSES,491,491,491,491,491,491,491,491
SUICIDE,508,508,508,508,508,508,508,508


In [21]:
# The 0.1% of the data would be:
len(train)*0.001

878.049

In [22]:
train.groupby('Category').count().sort_values('Dates').head(10)

,Dates,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
Category,,,,,,,,
TREA,6,6,6,6,6,6,6,6
PORNOGRAPHY/OBSCENE MAT,22,22,22,22,22,22,22,22
GAMBLING,146,146,146,146,146,146,146,146
SEX OFFENSES NON FORCIBLE,148,148,148,148,148,148,148,148
EXTORTION,256,256,256,256,256,256,256,256
BRIBERY,289,289,289,289,289,289,289,289
BAD CHECKS,406,406,406,406,406,406,406,406
FAMILY OFFENSES,491,491,491,491,491,491,491,491
SUICIDE,508,508,508,508,508,508,508,508


In [23]:
len(train["Category"].unique())

39

### 2. Load Test data

In [24]:
test = pd.read_csv(f'{PATH}test.csv',date_parser='Dates')

In [25]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


### 3. Merge the data

In [27]:
train.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')

In [28]:
test.columns

Index(['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y'], dtype='object')

In [44]:
# Let's remove extra columns
train.drop(['Descript','Resolution'],axis=1,inplace=True)
test.drop(['Id'],axis=1,inplace=True)
# Add dummy column in test dataset, which we will remove later
test['Category']='TREA'
# Add dataset identifier
train['is_train'] = 1
test['is_train'] = 0

In [141]:
#test['Category']='TREA'

In [142]:
train.columns

Index(['Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y',
       'is_train'],
      dtype='object')

In [143]:
test.columns

Index(['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Category',
       'is_train'],
      dtype='object')

In [144]:
df_master = pd.concat([train,test],axis=0,sort=True,ignore_index=True)

In [145]:
df_master.head()

,Address,Category,Dates,DayOfWeek,PdDistrict,X,Y,is_train
0,OAK ST / LAGUNA ST,WARRANTS,2015-05-13 23:53:00,Wednesday,NORTHERN,-122.425892,37.774599,1
1,OAK ST / LAGUNA ST,OTHER OFFENSES,2015-05-13 23:53:00,Wednesday,NORTHERN,-122.425892,37.774599,1
2,VANNESS AV / GREENWICH ST,OTHER OFFENSES,2015-05-13 23:33:00,Wednesday,NORTHERN,-122.424363,37.800414,1
3,1500 Block of LOMBARD ST,LARCENY/THEFT,2015-05-13 23:30:00,Wednesday,NORTHERN,-122.426995,37.800873,1
4,100 Block of BRODERICK ST,LARCENY/THEFT,2015-05-13 23:30:00,Wednesday,PARK,-122.438738,37.771541,1


In [146]:
df_master.tail()

,Address,Category,Dates,DayOfWeek,PdDistrict,X,Y,is_train
1762306,2600 Block of BRYANT ST,TREA,2003-01-01 00:01:00,Wednesday,MISSION,-122.408983,37.751987,0
1762307,1900 Block of WASHINGTON ST,TREA,2003-01-01 00:01:00,Wednesday,NORTHERN,-122.425342,37.792681,0
1762308,5500 Block of MISSION ST,TREA,2003-01-01 00:01:00,Wednesday,INGLESIDE,-122.445418,37.712075,0
1762309,1500 Block of HUDSON AV,TREA,2003-01-01 00:01:00,Wednesday,BAYVIEW,-122.387394,37.739479,0
1762310,1500 Block of SLOAT BL,TREA,2003-01-01 00:01:00,Wednesday,TARAVAL,-122.489714,37.733950,0


In [147]:
len(train), len(test), len(df_master)

(878049, 884262, 1762311)

### 4. Feature Engineering

In [148]:
df_master.columns

Index(['Address', 'Category', 'Dates', 'DayOfWeek', 'PdDistrict', 'X', 'Y',
       'is_train'],
      dtype='object')

#### Add data-time features

In [149]:
add_datepart(df=df_master,fldname='Dates',time=True)

In [150]:
df_master.columns

Index(['Address', 'Category', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'is_train',
       'DatesYear', 'DatesMonth', 'DatesWeek', 'DatesDay', 'DatesDayofweek',
       'DatesDayofyear', 'DatesIs_month_end', 'DatesIs_month_start',
       'DatesIs_quarter_end', 'DatesIs_quarter_start', 'DatesIs_year_end',
       'DatesIs_year_start', 'DatesHour', 'DatesMinute', 'DatesSecond',
       'DatesElapsed'],
      dtype='object')

#### Add X, Y features

In [151]:
# Lets' created scaled transform of the X and Y coordinates
xy_scaler = StandardScaler()
xy_scaler.fit(df_master.loc[:,['X','Y']])
xy_scaled = xy_scaler.transform(df_master.loc[:,['X','Y']])

# PCA on X and Y, select top 2 components
xy_pca = PCA(n_components=2, whiten=True).fit_transform(xy_scaled)

# Rotation
rot45_X = .707* xy_scaled[:,1] + .707* xy_scaled[:,0] 
rot45_Y = .707* xy_scaled[:,1] - .707* xy_scaled[:,0]

rot30_X = (1.732/2)* xy_scaled[:,0] + (1./2)* xy_scaled[:,1]
rot30_Y = (1.732/2)* xy_scaled[:,1] - (1./2)* xy_scaled[:,0]

rot60_X = (1./2)* xy_scaled[:,0] + (1.732/2)* xy_scaled[:,1] 
rot60_Y = (1./2)* xy_scaled[:,1] - (1.732/2)* xy_scaled[:,0]

# Polar coordinates
radial_r = np.sqrt( np.power(xy_scaled[:,1],2) + np.power(xy_scaled[:,0],2) )
radial_phi = np.arctan2(xy_scaled[:,1], xy_scaled[:,0])

xy_scaled_df=pd.DataFrame(xy_scaled)
xy_pca_df=pd.DataFrame(xy_pca)

In [152]:
df_master['xy_scaled_1']    = xy_scaled_df[0]
df_master['xy_scaled_2']    = xy_scaled_df[1]
df_master['xy_pca_1']       = xy_pca_df[0]
df_master['xy_pca_2']       = xy_pca_df[1]
df_master['xy_x_rotate_30'] = rot30_X
df_master['xy_x_rotate_45'] = rot45_X
df_master['xy_x_rotate_60'] = rot60_X
df_master['xy_y_rotate_30'] = rot30_Y
df_master['xy_y_rotate_45'] = rot45_Y
df_master['xy_y_rotate_60'] = rot60_Y
df_master['xy_radial_r']    = radial_r
df_master['xy_radial_phi']  = radial_phi

#### Add Address features

In [153]:
# Create a feature for street intersection
df_master['add_contains_intersection']=df_master['Address'].str.contains('/')

# Create a feature for block or no-block in the address
add_upper=df_master['Address'].str.upper()
df_master['add_contains_block']=add_upper.str.contains('BLOCK')

# Create a feature to capture the data before and after the '/'
add_p1, add_p2 = df_master['Address'].str.split('/', 1).str

add_p1=add_p1.str.rstrip()
df_master['add_part1']=add_p1.str.lstrip()
add_p2=add_p2.str.rstrip()
df_master['add_part2']=add_p2.str.lstrip()

# Check if the split went well
df_master[['Address','add_part1','add_part2']].head(15)

,Address,add_part1,add_part2
0,OAK ST / LAGUNA ST,OAK ST,LAGUNA ST
1,OAK ST / LAGUNA ST,OAK ST,LAGUNA ST
2,VANNESS AV / GREENWICH ST,VANNESS AV,GREENWICH ST
3,1500 Block of LOMBARD ST,1500 Block of LOMBARD ST,NaN
4,100 Block of BRODERICK ST,100 Block of BRODERICK ST,NaN
5,0 Block of TEDDY AV,0 Block of TEDDY AV,NaN
6,AVALON AV / PERU AV,AVALON AV,PERU AV
7,KIRKWOOD AV / DONAHUE ST,KIRKWOOD AV,DONAHUE ST
8,600 Block of 47TH AV,600 Block of 47TH AV,NaN
9,JEFFERSON ST / LEAVENWORTH ST,JEFFERSON ST,LEAVENWORTH ST


In [154]:
# Type of the first part of address (ST, AV BL etc)
df_master['add_part1_type']=df_master['add_part1'].str[-2:]
np.unique(df_master['add_part1_type'])

array(['80', 'AL', 'AR', 'AV', 'AY', 'BL', 'CR', 'CT', 'DR', 'ER', 'EX', 'HY', 'LN', 'MS', 'NO', 'PL', 'PZ',
       'RD', 'RK', 'RW', 'ST', 'TI', 'TR', 'WK', 'WY'], dtype=object)

### 5. Format the data

In [186]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [190]:
#train_ordinal = train[['PdDistrict']]
#enc.fit(train_ordinal)
#train_one_hot = enc.transform(train_ordinal)

In [155]:
train_cats(df_master)

In [156]:
x_all, y_all, nas = proc_df(df_master,'Category')

In [157]:
x_all.head()

,Address,DayOfWeek,PdDistrict,X,Y,is_train,DatesYear,DatesMonth,DatesWeek,DatesDay,...,xy_y_rotate_30,xy_y_rotate_45,xy_y_rotate_60,xy_radial_r,xy_radial_phi,add_contains_intersection,add_contains_block,add_part1,add_part2,add_part1_type
0,20896,7,5,-122.425892,37.774599,1,2015,5,20,13,...,0.058921,0.079634,0.094941,0.105766,3.074315,True,False,12436,956,21
1,20896,7,5,-122.425892,37.774599,1,2015,5,20,13,...,0.058921,0.079634,0.094941,0.105766,3.074315,True,False,12436,956,21
2,24170,7,5,-122.424363,37.800414,1,2015,5,20,13,...,0.081456,0.083140,0.079179,0.083269,2.303354,True,False,12899,749,4
3,4419,7,5,-122.426995,37.800873,1,2015,5,20,13,...,0.125209,0.144503,0.153987,0.154851,2.723442,False,True,4273,0,21
4,1924,7,6,-122.438738,37.771541,1,2015,5,20,13,...,0.262709,0.371149,0.454391,0.524343,3.140409,False,True,1924,0,21


In [158]:
x_all.tail()

,Address,DayOfWeek,PdDistrict,X,Y,is_train,DatesYear,DatesMonth,DatesWeek,DatesDay,...,xy_y_rotate_30,xy_y_rotate_45,xy_y_rotate_60,xy_radial_r,xy_radial_phi,add_contains_intersection,add_contains_block,add_part1,add_part2,add_part1_type
1762306,7920,7,4,-122.408983,37.751987,0,2003,1,1,1,...,-0.258273,-0.344039,-0.406447,0.447603,-0.091471,False,True,6909,0,21
1762307,5597,7,5,-122.425342,37.792681,0,2003,1,1,1,...,0.083205,0.094108,0.098622,0.098723,2.662674,False,True,5154,0,21
1762308,11613,7,3,-122.445418,37.712075,0,2003,1,1,1,...,0.262302,0.435898,0.579900,0.752704,-2.973937,False,True,9826,0,21
1762309,4394,7,1,-122.387394,37.739479,0,2003,1,1,1,...,-0.633203,-0.860450,-1.029278,1.151584,-0.058592,False,True,4248,0,4
1762310,4477,7,9,-122.489714,37.733950,0,2003,1,1,1,...,1.024599,1.489756,1.853770,2.187757,-3.105396,False,True,4331,0,6


In [159]:
train_x = pd.DataFrame(x_all[x_all['is_train']==1])
train_y = y_all[x_all['is_train']==1]
train_x.shape, len(train_y)

((878049, 39), 878049)

In [160]:
train_x.tail()

,Address,DayOfWeek,PdDistrict,X,Y,is_train,DatesYear,DatesMonth,DatesWeek,DatesDay,...,xy_y_rotate_30,xy_y_rotate_45,xy_y_rotate_60,xy_radial_r,xy_radial_phi,add_contains_intersection,add_contains_block,add_part1,add_part2,add_part1_type
878044,16537,2,9,-122.459033,37.714056,1,2003,1,2,6,...,0.487887,0.752699,0.966408,1.192232,-3.039591,True,False,11863,315,21
878045,11841,2,3,-122.447364,37.731948,1,2003,1,2,6,...,0.330543,0.510560,0.655915,0.809880,-3.038404,False,True,10017,0,21
878046,11663,2,8,-122.403390,37.780266,1,2003,1,2,6,...,-0.297467,-0.430523,-0.534350,0.628375,0.030463,True,False,9854,657,21
878047,23739,2,8,-122.390531,37.780607,1,2003,1,2,6,...,-0.506459,-0.726414,-0.897050,1.047512,0.018964,True,False,12853,43,21
878048,5301,2,1,-122.394926,37.738212,1,2003,1,2,6,...,-0.512757,-0.688746,-0.817975,0.906771,-0.077413,False,True,4936,0,4


In [161]:
train_y[1:50], np.unique(train_y)

(array([21, 21, 16, 16, 16, 36, 36, 16, 16, 16, 21, 35, 16, 20, 20, 25,  1, 21, 20, 16, 25, 37, 20, 16, 20,
        16, 16, 16, 21, 16, 20, 35, 16, 35, 16, 16, 16, 38, 35, 20, 16, 16, 21, 21, 21, 36, 16, 20,  4],
       dtype=int8),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
        25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], dtype=int8))

In [162]:
test_x = pd.DataFrame(x_all[x_all['is_train']==0])
test_x.shape

(884262, 39)

In [163]:
test_x.tail()

,Address,DayOfWeek,PdDistrict,X,Y,is_train,DatesYear,DatesMonth,DatesWeek,DatesDay,...,xy_y_rotate_30,xy_y_rotate_45,xy_y_rotate_60,xy_radial_r,xy_radial_phi,add_contains_intersection,add_contains_block,add_part1,add_part2,add_part1_type
1762306,7920,7,4,-122.408983,37.751987,0,2003,1,1,1,...,-0.258273,-0.344039,-0.406447,0.447603,-0.091471,False,True,6909,0,21
1762307,5597,7,5,-122.425342,37.792681,0,2003,1,1,1,...,0.083205,0.094108,0.098622,0.098723,2.662674,False,True,5154,0,21
1762308,11613,7,3,-122.445418,37.712075,0,2003,1,1,1,...,0.262302,0.435898,0.579900,0.752704,-2.973937,False,True,9826,0,21
1762309,4394,7,1,-122.387394,37.739479,0,2003,1,1,1,...,-0.633203,-0.860450,-1.029278,1.151584,-0.058592,False,True,4248,0,4
1762310,4477,7,9,-122.489714,37.733950,0,2003,1,1,1,...,1.024599,1.489756,1.853770,2.187757,-3.105396,False,True,4331,0,6


################################ New Development ################################

In [200]:
train_x.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'is_train', 'DatesYear',
       'DatesMonth', 'DatesWeek', 'DatesDay', 'DatesDayofweek',
       'DatesDayofyear', 'DatesIs_month_end', 'DatesIs_month_start',
       'DatesIs_quarter_end', 'DatesIs_quarter_start', 'DatesIs_year_end',
       'DatesIs_year_start', 'DatesHour', 'DatesMinute', 'DatesSecond',
       'DatesElapsed', 'xy_scaled_1', 'xy_scaled_2', 'xy_pca_1', 'xy_pca_2',
       'xy_x_rotate_30', 'xy_x_rotate_45', 'xy_x_rotate_60', 'xy_y_rotate_30',
       'xy_y_rotate_45', 'xy_y_rotate_60', 'xy_radial_r', 'xy_radial_phi',
       'add_contains_intersection', 'add_contains_block', 'add_part1',
       'add_part2', 'add_part1_type'],
      dtype='object')

In [201]:
train_ordinal = train_x[['PdDistrict','DatesDayofweek']]
enc = preprocessing.LabelEncoder()
enc.fit(train_ordinal)
new_cat_features = enc.transform(train_ordinal)
new_cat_features

ValueError: bad input shape (878049, 2)

In [203]:
cat_features = ['PdDistrict','DatesDayofweek']
enc = preprocessing.LabelEncoder()
enc.fit(cat_features)
new_cat_features = enc.transform(cat_features)
print(new_cat_features) # [1 2 0]

[1 0]


In [205]:
new_cat_features = new_cat_features.reshape(-1, 1) # Needs to be the correct shape
ohe = preprocessing.OneHotEncoder(sparse=False) #Easier to read
tt=ohe.fit_transform(new_cat_features)
tt

array([[0., 1.],
       [1., 0.]])

In [206]:
len(tt), new_cat_features

(2, array([[1],
        [0]]))

In [196]:
ohe = preprocessing.OneHotEncoder(sparse=False) #Easier to read
print(ohe.fit_transform(new_cat_features))

ValueError: Expected 2D array, got 1D array instead:
array=[4. 4. 4. ... 7. 7. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### 6. Building the models

In [164]:
# Base model of 30 trees
m = RandomForestClassifier(n_jobs=-1,n_estimators=30)
m.fit(train_x, train_y)
m.score(train_x, train_y)

0.8574373411962203

In [165]:
# Adding max_features (columns)
m2 = RandomForestClassifier(n_jobs=-1,n_estimators=30,max_features=0.6)
m2.fit(train_x, train_y)
m2.score(train_x, train_y)

0.8574441745278453

In [166]:
rare_cats = set(['SUICIDE','FAMILY OFFENSES', 'BAD CHECKS', 'BRIBERY', 'EXTORTION',
       'SEX OFFENSES NON FORCIBLE', 'GAMBLING', 'PORNOGRAPHY/OBSCENE MAT',
       'TREA'])
all_cats = set(train['Category'].unique())
common_cats = all_cats-rare_cats
# Create index of common Categories
common_cat_idx = train['Category'].isin(common_cats)
# subset train data
train_x_common_cats = train_x[common_cat_idx]
train_y_common_cats = train_y[common_cat_idx]
len(train_x_common_cats), len(train_y_common_cats)

(875777, 875777)

In [167]:
# Using common features only
m3 = RandomForestClassifier(n_jobs=-1,n_estimators=30)
m3.fit(train_x_common_cats, train_y_common_cats)
m3.score(train_x_common_cats, train_y_common_cats)

0.8583520690769454

In [168]:
# Using common features only and min_samples (tree stops when # examples equals to)
m4 = RandomForestClassifier(n_jobs=-1,n_estimators=30, min_samples_leaf=3)
m4.fit(train_x_common_cats, train_y_common_cats)
m4.score(train_x_common_cats, train_y_common_cats)

0.7442762255688378

In [169]:
# Using common features only and max_feature (max percent of columns in any given tree)
m5 = RandomForestClassifier(n_jobs=-1,n_estimators=30, max_features=0.5)
m5.fit(train_x_common_cats, train_y_common_cats)
m5.score(train_x_common_cats, train_y_common_cats)

0.8583178137813622

In [170]:
# Using common features only and max_feature (max percent of columns in any given tree)
m6 = RandomForestClassifier(n_jobs=-1,n_estimators=40, max_features=0.5, min_samples_leaf=3)
m6.fit(train_x_common_cats, train_y_common_cats)
m6.score(train_x_common_cats, train_y_common_cats)

0.7947445525516199

### 7. Testing the models

In [171]:
results_m1 = m.predict(test_x)

In [175]:
results_m2 = m2.predict(test_x)

In [176]:
results_m3 = m3.predict(test_x)

In [177]:
results_m4 = m4.predict(test_x)

In [178]:
results_m5 = m5.predict(test_x)

In [179]:
results_m6 = m6.predict(test_x)

### 8. Writing the Output

In [172]:
def write_results(results,file_name):
    
    result_accumulator=list()
    print("Step 1: one-hot-encoding...")

    for i in range(0,len(results)):
        temp=np.zeros(39,dtype=int)
        temp[results[i]]=1
        line=np.insert(temp,0,i)
        result_accumulator.append(line)

    print("Step 2: Formatting into dataframe...")    

    out_file = pd.DataFrame(result_accumulator)
    print("Dimension of the output file: ",out_file.shape)

    print("Step 3: Writing the output file...") 
    out_file_address = 'data/sf-crime/' + file_name + '.csv'
    out_file.to_csv(out_file_address,index=False)

In [173]:
write_results(results_m1,'results_m1')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


In [180]:
write_results(results_m2,'results_m2')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


In [181]:
write_results(results_m3,'results_m3')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


In [182]:
write_results(results_m4,'results_m4')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


In [183]:
write_results(results_m5,'results_m5')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


In [184]:
write_results(results_m6,'results_m6')

Step 1: one-hot-encoding...
Step 2: Formatting into dataframe...
Dimension of the output file:  (884262, 40)
Step 3: Writing the output file...


### 9. Results based on kaggle evaluation script

model - m1 ; score - 24.68536
#model - m2 ; score - TBS
model - m3 ; score - 24.65935
model - m4 ; score - 24.04209
#model - m5 ; score - TBS
model - m6 ; score - 24.01581

### 10. What to try next?

#### More features:

- Weekday vs weekend
- Year-month combo

#### Methodology:

- One hot encoding for categorical variables?

#### Other models:

- NeuralNet
- LightGBM
- XGBoost